In [1]:
%autosave 5

Autosaving every 5 seconds


In [14]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [15]:
import pandas as pd 
MultiTopicDocs = pd.read_csv("/content/drive/My Drive/AdvancedNLPProject/Sravani/data-4topics500.csv")
MultiTopicDocs.head()

,Unnamed: 0,Doc,Num_of_topics,Topics,Topic_order
0,0,Black students were hurt most when Wellesley t...,3,"['PoliticsPolicy', 'Health', 'Food']","[2, 2, 0, 2, 3, 0, 3, 2, 2, 2, 0, 2, 0, 0, 0, ..."
1,1,BACKGROUND.\nant colony algorithm belongs to ...,3,"['Food', 'Health', 'Sports']","[0, 0, 3, 1, 3, 3, 3, 3, 1, 1, 0, 1, 3, 3, 3, ..."
2,2,BACKGROUND.\n9 myths about sexual assault.\nlo...,4,"['PoliticsPolicy', 'Health', 'Food', 'Sports']","[0, 2, 0, 0, 2, 2, 1, 0, 2, 0, 2, 3, 0, 3, 3, ..."
3,3,Jockey Ramon Dominguez has made enough progres...,2,"['Sports', 'Food']","[1, 3, 3, 1, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, ..."
4,4,Love it!I really like the flavor of this one!\...,2,"['Food', 'PoliticsPolicy']","[3, 2, 2, 3, 2, 2, 3, 3, 3, 2, 3, 3, 3, 3, 2, ..."


In [16]:
'''Unzipping Files '''
# import os 
# os.chdir('/content/drive/My Drive/AdvancedNLPProject/Sravani/Data')
# !unzip -u "/content/drive/My Drive/AdvancedNLPProject/Sravani/Data/glove.840B.300d.zip" -d "/content/drive/My Drive/AdvancedNLPProject/Sravani/Data"

'Unzipping Files '

In [17]:
import re

def clean_text(x):
    pattern = r'[^a-zA-z0-9\s]'
    text = re.sub(pattern, '', x)
    return x

def clean_numbers(x):
    if bool(re.search(r'\d', x)):
        x = re.sub('[0-9]{5,}', '', x)
        x = re.sub('[0-9]{4}', '', x)
        x = re.sub('[0-9]{3}', '', x)
        x = re.sub('[0-9]{2}', '', x)
    return x

In [18]:
contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}
def _get_contractions(contraction_dict):
    contraction_re = re.compile('(%s)' % '|'.join(contraction_dict.keys()))
    return contraction_dict, contraction_re
contractions, contractions_re = _get_contractions(contraction_dict)
def replace_contractions(text):
    def replace(match):
        return contractions[match.group(0)]
    return contractions_re.sub(replace, text)

In [19]:
AllTextFiles = list(MultiTopicDocs['Doc'])
data=""
for f in AllTextFiles:
    data+=f

Sentences = []
for line in data.split('.'):
  # lower the text
  line = line.lower()

  # Clean the text
  line = clean_text(line)

  # Clean numbers
  line = clean_numbers(line)

  # Clean Contractions
  line = replace_contractions(line)

  Sentences.append(line)

print(len(Sentences),Sentences[0])

170880 black students were hurt most when wellesley tried to control grade inflation


In [20]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

max_features = 120000 # how many unique words to use (i.e num rows in embedding vector) . This is also the vocabulary size 
maxlen = 750 # max number of words in a question to use

## Tokenize the sentences
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(Sentences)
Sentences = tokenizer.texts_to_sequences(Sentences)

## Pad the sentences 
Sentences = pad_sequences(Sentences, maxlen=maxlen)

In [21]:
import numpy as np 
def load_glove(word_index):
    EMBEDDING_FILE = '/content/drive/My Drive/AdvancedNLPProject/Sravani/Data/glove.840B.300d.txt'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')[:300]
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))
    
    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = -0.005838499,0.48782197
    embed_size = all_embs.shape[1]

    nb_words = min(max_features, len(word_index)+1)
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: 
            embedding_matrix[i] = embedding_vector
        else:
            embedding_vector = embeddings_index.get(word.capitalize())
            if embedding_vector is not None: 
                embedding_matrix[i] = embedding_vector
    return embedding_matrix

embedding = 1
if embedding :
  embedding_matrix = load_glove(tokenizer.word_index)
else :
  embedding_matrix = np.random.randn(120000,300)
np.shape(embedding_matrix)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  exec(code_obj, self.user_global_ns, self.user_ns)


(41363, 300)

In [22]:
word2idx = tokenizer.word_index
print(len(word2idx))

41362


In [23]:
import re
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from sklearn.cluster import KMeans 
from sklearn.metrics import silhouette_samples, silhouette_score

Predicted_NoOfTopics=[]
Actual_NoOfTopics=[]
for index, row in MultiTopicDocs.iterrows():
    Document= row['Doc'] # Current document 
    Actual_NoOfTopics.append(int(row['Num_of_topics']))
    words = word_tokenize(Document) 
    unique_words = []
    for word in words:
      if words not in unique_words:
        unique_words.append(word)

    current_embedding_layer = []
    for word in unique_words :
      if word in word2idx.keys():
        wordidx= int(word2idx[word])
        current_embedding_layer.append(embedding_matrix[wordidx])

    no_of_clusters = [2,3,4] 
    silhotte_score_array =[]
    for n_clusters in no_of_clusters: 
      cluster = KMeans(n_clusters = n_clusters) 
      cluster_labels = cluster.fit_predict(current_embedding_layer) 
  
      # The silhouette_score gives the  
      # average value for all the samples. 
      silhouette_avg = silhouette_score(current_embedding_layer, cluster_labels) 
      silhotte_score_array.append(silhouette_avg)
   
    Predicted_NoOfTopics.append(np.argmax(silhotte_score_array)+2)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [24]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

print("Accuracy: ",accuracy_score(Actual_NoOfTopics,Predicted_NoOfTopics))
print("F1Score(Macro) :",f1_score(Actual_NoOfTopics,Predicted_NoOfTopics, average='macro'))

Accuracy:  0.322
F1Score(Macro) : 0.21840753767892554


In [25]:
print(Actual_NoOfTopics)
print(Predicted_NoOfTopics)

[3, 3, 4, 2, 2, 2, 2, 4, 4, 3, 2, 3, 4, 3, 4, 2, 4, 3, 2, 4, 4, 3, 2, 2, 4, 2, 2, 2, 4, 2, 3, 2, 4, 4, 4, 4, 4, 2, 3, 2, 2, 4, 3, 2, 3, 2, 4, 2, 3, 2, 4, 3, 2, 2, 2, 3, 2, 4, 4, 2, 4, 4, 3, 4, 3, 3, 4, 4, 2, 3, 4, 3, 2, 3, 2, 2, 4, 2, 3, 4, 2, 2, 3, 3, 2, 2, 4, 3, 4, 2, 4, 2, 2, 2, 2, 3, 4, 3, 3, 4, 3, 3, 4, 3, 4, 3, 2, 4, 3, 2, 2, 2, 4, 2, 4, 2, 4, 2, 3, 3, 3, 2, 3, 2, 4, 4, 2, 2, 3, 4, 3, 4, 4, 2, 3, 2, 4, 3, 4, 4, 3, 4, 4, 3, 3, 3, 3, 4, 2, 2, 2, 3, 2, 2, 3, 2, 4, 2, 3, 2, 3, 3, 3, 4, 4, 3, 3, 3, 4, 3, 2, 4, 3, 4, 4, 2, 3, 4, 4, 2, 4, 2, 3, 2, 3, 2, 2, 4, 4, 2, 4, 4, 2, 2, 3, 3, 3, 2, 4, 4, 3, 3, 4, 3, 4, 2, 4, 3, 4, 4, 3, 3, 3, 2, 4, 2, 3, 4, 2, 2, 4, 4, 2, 4, 3, 4, 2, 2, 2, 3, 3, 4, 3, 2, 2, 3, 2, 2, 2, 2, 3, 4, 2, 2, 2, 3, 3, 4, 4, 4, 3, 3, 3, 3, 2, 4, 4, 2, 3, 3, 4, 3, 4, 2, 3, 3, 4, 4, 3, 4, 3, 3, 4, 4, 3, 4, 3, 4, 3, 2, 2, 3, 2, 2, 2, 2, 2, 4, 4, 3, 4, 2, 3, 3, 3, 4, 3, 2, 2, 4, 2, 4, 2, 2, 4, 2, 3, 3, 3, 2, 3, 3, 2, 2, 3, 4, 4, 3, 4, 4, 2, 4, 3, 4, 3, 4, 2, 3, 3, 3, 4, 2, 2, 